In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
def get_yahoo_news(ticker):

    
    articles = []
    for item in soup.find_all('li', class_='js-stream-content'):
        headline = item.find('h3')
        if headline:
            title = headline.text
            link = 'https://finance.yahoo.com' + headline.find('a')['href']
            # Try to find timestamp
            try:
                timestamp = item.find('time')['datetime']
            except:
                timestamp = None
            articles.append({
                'ticker': ticker,
                'title': title,
                'url': link,
                'timestamp': timestamp
            })
    
    return articles

# Example for TSLA
tsla_articles = get_yahoo_news('TSLA')

# Convert to DataFrame
news_df = pd.DataFrame(tsla_articles)

# View
print(news_df.head())

Empty DataFrame
Columns: []
Index: []


In [3]:
ticker = 'TSLA'
url = f'https://finance.yahoo.com/quote/{ticker}/news?p={ticker}'
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [32]:
f'https://news.search.yahoo.com/search?p={ticker}'

'https://news.search.yahoo.com/search?p=TSLA'

In [28]:
import re
import csv
from time import sleep
from bs4 import BeautifulSoup
import requests

headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.google.com',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36 Edg/85.0.564.44'
}

def get_article(card):
    """Extract article information from the raw html"""
    headline = card.find('h4', 's-title').text
    source = card.find("span", 's-source').text
    posted = card.find('span', 's-time').text.replace('·', '').strip()
    description = card.find('p', 's-desc').text.strip()
    raw_link = card.find('a').get('href')
    unquoted_link = requests.utils.unquote(raw_link)
    pattern = re.compile(r'RU=(.+)\/RK')

    clean_link = re.search(pattern, unquoted_link).group(1)

        
    article = (headline, source, posted, description, clean_link)
    return article

def get_the_news(ticker):
    """Run the main program"""
    url = f'https://news.search.yahoo.com/search?p={ticker}'
    articles = []
    links = set()
    
    while True:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'NewsArticle')

        # extract articles from page
        for card in cards:
            try:
                article = get_article(card)
            except:
                pass

            link = article[-1]
            if not link in links:
                links.add(link)
                articles.append(article)        
                
        # find the next page
        try:
            url = soup.find('a', 'next').get('href')
            sleep(1)
        except AttributeError:
            break
        
    return articles


In [29]:

df = get_the_news('TSLA')


In [30]:
df

[('Tesla, Inc. (TSLA) Bets on Full Self-Driving and Optimus Robots as AI Future...',
  'Insider Monkey ·  via Yahoo Finance',
  '16 hours ago',
  'We recently published a list of 10 AI Stocks That Will Skyrocket. In this article, we are going to...',
  'https://finance.yahoo.com/news/tesla-inc-tsla-bets-full-121650688.html?fr=sycsrp_catchall'),
 ('Why Tesla Inc. (TSLA) Soared Last Week',
  'Insider Monkey ·  via Yahoo Finance',
  '8 hours ago',
  'We recently published a list of Why These 10 Firms Recorded Double-Digit Gains Last Week. In this article, we are going to take a look at where Tesla...',
  'https://finance.yahoo.com/news/why-tesla-inc-tsla-soared-180528904.html?fr=sycsrp_catchall'),
 ('Tesla (TSLA) Shares Climb Despite Decline in Deliveries',
  'Guru Focus',
  '10 hours ago',
  "Tesla (TSLA) shares rose despite a dip in deliveries and profit. Elon Musk's commitment to focusing on Tesla boosts investor confidence. Analysts off",
  'https://www.gurufocus.com/news/2806595/tesl